In [1]:
!pip install krwordrank
!pip install kiwipiepy

In [53]:
import pandas as pd

df = pd.read_csv('./2022-04-18 news data.csv')
df = df[df['카테고리']=='정치']
df

,Unnamed: 0,카테고리,언론사,기사제목,날짜,본문,기사링크
0,0,정치,SBS,[단독] '6억 집을 1억에'…이창용 한은 총재 후보자 다운계약 의혹,2022.04.18.,"\n\n\n\n\n이창용 한국은행 총재 후보자가 지난 2002년, 서울 중구 신당동...",https://news.naver.com//main/read.naver?mode=L...
1,1,정치,중앙일보,"노정희 선관위원장, '소쿠리 대선' 논란 44일만에 사퇴",2022.04.18.,\n\n\n\n\n\t\n\t 노정희 중앙선거관리위원회위원장이 18일 사의를 표명...,https://news.naver.com//main/read.naver?mode=L...
2,2,정치,YTN,단거리 발사에 김정은 참석...전술핵 탑재 노렸나,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]김정은 국무위원장이 단...,https://news.naver.com//main/read.naver?mode=L...
3,3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...
4,4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...
...,...,...,...,...,...,...,...
884,884,정치,중앙일보,검수완박 핵심은 부칙 2조…검찰이 하던 정권수사 올스톱,2022.04.18.,\n\n\n\n\n\t\n\t “이 법 시행 당시 검찰에(서) 수사 계속 중인 사...,https://news.naver.com//main/read.naver?mode=L...
885,885,정치,중앙일보,당선인 “팩트 확실해야” 했지만…정호영 부실 검증론 확산,2022.04.18.,\n\n\n\n\n\t\n\t 정호영 보건복지부 장관 후보자를 둘러싼 각종 의혹이...,https://news.naver.com//main/read.naver?mode=L...
886,886,정치,조선일보,"“尹 길 걸으시라”던 조성은, 김오수 사퇴에 “본인 운명 어쩌겠나”",2022.04.18.,\n\n\n\n\n김오수 검찰총장이 검찰수사권 완전박탈 입법 추진의 부당성을 호소하...,https://news.naver.com//main/read.naver?mode=L...
887,887,정치,더팩트,"文대통령, '백서, 책, 영상' 남기며 퇴임 준비 차근차근 진행",2022.04.18.,\n\n\n\n\n'친구이자 동지' 노무현 전 대통령과 유사한 행보오는 5월 9일 ...,https://news.naver.com//main/read.naver?mode=L...


In [54]:
from krwordrank.word import summarize_with_keywords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
from kiwipiepy import Kiwi, Match
from kiwipiepy.utils import Stopwords

def clean_text(title):
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※%~∼ㆍ!【】』㈜©囹圄秋 ■◆◇▷▶◁◀ △▲▽▼<>‘|\(\)\[\]`\'…》→←↑↓↔〓♤♠♡♥♧♣⊙◈▣◐◑☆★\”\“\’·※~ ! @ # $ % ^ & * \ " ]', ' ', title)
    text_rmv = ' '.join(text_rmv.split())
    return text_rmv # 문자열

def noun_extractor(sentence):
    nouns =[]
    kiwi = Kiwi()
    stopwords = Stopwords()
    result_iter = kiwi.tokenize(sentence, normalize_coda=True,stopwords=stopwords)
    i = 0
    for i in range(len(result_iter)):
        if result_iter[i][1] == 'NNP':
            nouns.append(result_iter[i][0])
    return nouns # 리스트로 반환

def listToString(str_list): #리스트를 문자열로 반환 (특수문자 제거 후 추출된 명사 입력)
    result = ""
    for s in str_list:
        result += s + " "
    string = result.strip()
    return string

def kwordrank(string): #해당 함수에서 불용어 제거 
    stop_words_list=[]
    stop_words_file = open("korean_stop_words.txt",encoding='utf-8')
    for line in stop_words_file.readlines():
        stop_words_list.append(line.rstrip())
        stop_words_file.close()
    stopwords=stop_words_list
    texts = [string]
    keywords = summarize_with_keywords(texts, min_count=2, max_length=10,beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
    passwords = {word:score for word, score in sorted(
        keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}
    return passwords


In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import numpy as np

def show_relevant_article(df): #df.본문

    # 기사 본문 클러스터링
    text = list(df.본문)

    lst = []
    for i in range(len(text)):
      text[i] = clean_text(text[i])
      lst.append(text[i])
    
    df['cleaned_본문'] = lst

    content = df['cleaned_본문'].tolist()
    n_clusters = 30 # 군집화할 갯수
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(content)
    X = normalize(X)
    kmeans = KMeans(n_clusters=n_clusters).fit(X)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    df['labels'] = labels
    data = df.labels # 숫자 라벨 생성
    labels_count = data.value_counts()
    
    # kwrodrank로 라벨 키워드 뽑기 
    top_word_30 = []
    for i in range(len(labels_count)):
      df_i = df[df.labels==i]

      df_i_content = list(df_i.cleaned_본문)
      df_content_del = listToString(df_i_content)

      k = kwordrank(df_content_del)
      k_1 = next(iter(k))
      top_word_30.append(k_1)

    # 숫자 라벨과 kwordrank로 추출한 키워드를 replace
    for i in range(len(labels_count)):
      for j in range(len(data)):
        if i == data[j]:
          data[j] = top_word_30[i]

    df = df.drop('cleaned_본문', axis = 1)

    return df


In [59]:
show_relevant_article(df)

scan vocabs ... 
num vocabs = 1208
done
scan vocabs ... 
num vocabs = 7578
done
scan vocabs ... 
num vocabs = 977
done
scan vocabs ... 
num vocabs = 2661
done
scan vocabs ... 
num vocabs = 3695
done
scan vocabs ... 
num vocabs = 4514
done
scan vocabs ... 
num vocabs = 3918
done
scan vocabs ... 
num vocabs = 4595
done
scan vocabs ... 
num vocabs = 4552
done
scan vocabs ... 
num vocabs = 6739
done
scan vocabs ... 
num vocabs = 6880
done
scan vocabs ... 
num vocabs = 4837
done
scan vocabs ... 
num vocabs = 2997
done
scan vocabs ... 
num vocabs = 13471
done
scan vocabs ... 
num vocabs = 7256
done
scan vocabs ... 
num vocabs = 2172
done
scan vocabs ... 
num vocabs = 5417
done
scan vocabs ... 
num vocabs = 3517
done
scan vocabs ... 
num vocabs = 18488
done
scan vocabs ... 
num vocabs = 4316
done
scan vocabs ... 
num vocabs = 2448
done
scan vocabs ... 
num vocabs = 9871
done
scan vocabs ... 
num vocabs = 2205
done
scan vocabs ... 
num vocabs = 1634
done
scan vocabs ... 
num vocabs = 7839
done

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Unnamed: 0,카테고리,언론사,기사제목,날짜,본문,기사링크,labels
0,0,정치,SBS,[단독] '6억 집을 1억에'…이창용 한은 총재 후보자 다운계약 의혹,2022.04.18.,"\n\n\n\n\n이창용 한국은행 총재 후보자가 지난 2002년, 서울 중구 신당동...",https://news.naver.com//main/read.naver?mode=L...,후보자
1,1,정치,중앙일보,"노정희 선관위원장, '소쿠리 대선' 논란 44일만에 사퇴",2022.04.18.,\n\n\n\n\n\t\n\t 노정희 중앙선거관리위원회위원장이 18일 사의를 표명...,https://news.naver.com//main/read.naver?mode=L...,대통령
2,2,정치,YTN,단거리 발사에 김정은 참석...전술핵 탑재 노렸나,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]김정은 국무위원장이 단...,https://news.naver.com//main/read.naver?mode=L...,북한
3,3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...,후보자
4,4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...,국민의당
...,...,...,...,...,...,...,...,...
884,884,정치,중앙일보,검수완박 핵심은 부칙 2조…검찰이 하던 정권수사 올스톱,2022.04.18.,\n\n\n\n\n\t\n\t “이 법 시행 당시 검찰에(서) 수사 계속 중인 사...,https://news.naver.com//main/read.naver?mode=L...,검찰
885,885,정치,중앙일보,당선인 “팩트 확실해야” 했지만…정호영 부실 검증론 확산,2022.04.18.,\n\n\n\n\n\t\n\t 정호영 보건복지부 장관 후보자를 둘러싼 각종 의혹이...,https://news.naver.com//main/read.naver?mode=L...,후보자
886,886,정치,조선일보,"“尹 길 걸으시라”던 조성은, 김오수 사퇴에 “본인 운명 어쩌겠나”",2022.04.18.,\n\n\n\n\n김오수 검찰총장이 검찰수사권 완전박탈 입법 추진의 부당성을 호소하...,https://news.naver.com//main/read.naver?mode=L...,검찰
887,887,정치,더팩트,"文대통령, '백서, 책, 영상' 남기며 퇴임 준비 차근차근 진행",2022.04.18.,\n\n\n\n\n'친구이자 동지' 노무현 전 대통령과 유사한 행보오는 5월 9일 ...,https://news.naver.com//main/read.naver?mode=L...,대통령
